### Code Hist.

 - CODE : Resample - KIER Energy Usage  
 - DESC  
    &ensp; 1) "M02-01_Data_06_Restore_Energy ACCU.ipynb"로부터 복원된 에너지 적산 사용량을 Resample  
    &ensp; 2) Resample된 데이터를 기반으로 기간별(10MIN, 30MIN, 1H, 12H, 24H, 1W, 2W, 1M) 순시 사용 데이터 산출 및 CSV 추출  

 - DATE  
    &ensp; 2024-08-05 Created  

# 01. Code

## 01-01. Init

### 01-01-01. Init_Module Import

In [1]:
#region Import_Basic Module
## Basic
import os, sys, warnings
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.path.dirname(os.path.abspath('./__file__'))
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('./__file__'))))
warnings.filterwarnings('ignore')

import numpy as np, pandas as pd
from pandas import DataFrame, Series
pd.options.display.float_format = '{:.10f}'.format

import math, random

## Datetime
import time, datetime as dt
from datetime import datetime, date, timedelta

## glob
import glob, requests, json
from glob import glob

## 시각화
import matplotlib.pyplot as plt, seaborn as sns
# %matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from scipy import stats

## Split, 정규화
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# K-Means 알고리즘
from sklearn.cluster import KMeans, MiniBatchKMeans

# Clustering 알고리즘의 성능 평가 측도
from sklearn import metrics
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, silhouette_score, rand_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.metrics.cluster import contingency_matrix

## For Web
import urllib
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

import tqdm
from tqdm.notebook import tqdm
#endregion Import_Basic Module

In [2]:
## Import_DL
str_tar = "tf"
## For Torch
if str_tar == "torch":
    import torch
    import torch.nn as nn
    from torch.nn.utils import weight_norm
    print("Torch Imported")
## For TF
elif str_tar == "tf":
    import tensorflow as tf
    import tensorflow_addons as tfa
    print("Tensorflow Imported")
else:
    print("Error : Cannot be used except for Keywords")
    print(" : torch / tf")

Tensorflow Imported


In [3]:
## Import_Local
from Src_Dev_Common import Data_Datetime as com_date, Data_Preprocessing as com_Prep, KMA_Weather as com_KMA, KECO_AirKor as com_KECO, KASI_Holiday as com_Holi, KIER_Usage_M02 as com_KIER_M02, Data_Analysis as com_Analysis, Data_Visualization as com_Visual, KIER_Usage as com_KIER

### 01-01-02. Config (Directory, Params)

In [4]:
## Init_config
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"], os.environ['TF_DETERMINISTIC_OPS'] = str(SEED), "1"

In [5]:
## Define Todate str
str_now_ymd = pd.datetime.now().date()
str_now_y, str_now_m, str_now_d = pd.datetime.now().year, pd.datetime.now().month, pd.datetime.now().day
str_now_hr, str_now_min = pd.datetime.now().hour, pd.datetime.now().minute

print(pd.datetime.now())
print(str(str_now_y) + " / " + str(str_now_m)  + " / " + str(str_now_d))
print(str(str_now_hr) + " : " + str(str_now_min))

2024-08-07 11:03:17.498538
2024 / 8 / 7
11 : 3


In [6]:
## Dict_Domain
int_domain = 5

## Domain, ACCU/INST Column
str_domain, str_col_accu, str_col_inst = com_KIER_M02.create_domain_str(int_domain)
## Directory Root
str_dirData, str_dir_raw, str_dir_cleansed, str_dirName_bld, str_dirName_h = com_KIER_M02.create_dir_str(str_domain)

## File
str_fileRaw, str_fileRaw_hList = str('KIER_RAW_' + str_domain + '_H_ID_Adopted.csv'), str('KIER_hList_Common.csv')

print(str(os.listdir(str_dirData)) + "\n")
print(os.listdir(str_dirName_h))

5 : GAS
['.ipynb_checkpoints', 'BS_CONFIGURATION_202309251452.csv', 'DATE_1M_2023-10-20.csv', 'KIER 전처리 현황_2024-06-25.xlsx', 'KIER_0_Raw', 'KIER_1_Cleansed', 'KIER_2_BLD', 'KIER_3_H_ELEC', 'KIER_3_H_GAS', 'KIER_3_H_HEAT', 'KIER_3_H_HOT_FLOW', 'KIER_3_H_HOT_HEAT', 'KIER_3_H_WATER', 'KIER_ASOS_WEATHER_DAILY_202309251521.csv', 'KIER_ASOS_WEATHER_HOUR_202309251521.csv', 'KIER_DATA_OLD', 'KIER_ETC', 'KIER_hList_Comparison_2024-06-26.xlsx', 'KIER_List_Table_Column_2023-09-25.xlsx', 'KIER_Query_2023-09-25.txt', 'KMA_ASOS_119_2010_2023_1st_to CSV.csv', '[IITP] 데이터 테이블 정리 (공유 원본).docx']

['KIER_GAS_561-1-1_ACCU_01_Raw.csv', 'KIER_GAS_561-1-1_INST_01_10min.csv', 'KIER_GAS_561-1-2_ACCU_01_Raw.csv', 'KIER_GAS_561-1-2_INST_01_10min.csv', 'KIER_GAS_561-1-3_ACCU_01_Raw.csv', 'KIER_GAS_561-1-3_INST_01_10min.csv', 'KIER_GAS_561-1-4_ACCU_01_Raw.csv', 'KIER_GAS_561-1-4_INST_01_10min.csv', 'KIER_GAS_561-10-1_ACCU_01_Raw.csv', 'KIER_GAS_561-10-1_INST_01_10min.csv', 'KIER_GAS_561-10-2_ACCU_01_Raw.csv', 'KIE

## 01-02. Data Load (df_raw)

#### 02-01. KIER (hList)

In [7]:
## "KIER_01-01_Data_hList.ipynb"로부터 만들어진 Bld/F/H List
df_kier_hList = pd.read_csv(str_dir_raw + str_fileRaw_hList, index_col = 0)
# print(df_kier_hList.shape, ' /// ', df_kier_hList.columns)
# list(df_kier_hList['HOUSE_ID'])
# df_kier_hList['HOUSE_ID']

### 01-02-01. KIER (Energy Usage)

In [8]:
## 시간 간격 (10분 / 30분 / 1시간 / 1일 / 1주 / 1달) 
list_col_interval = ['10T', '30T', '1H', '12H', '1D', '1W', '2W', '1M']
# list_col_interval = ['10T', '1H', '12H', '1D', '1W', '2W', '1M']
# list_col_interval = ['1M']
for str_int in list_col_interval:
    print(str_int)
    if str_int == '10T' : 
        str_int = '10MIN'
        str_file_inst = 'KIER_' + str_domain + '_INST_10MIN_2st_Linear.csv'
        df_kier_inst = pd.read_csv(str_dirName_h + str_file_inst, index_col = 0)
        str_file_inst = 'KIER_' + str_domain + '_INST_' + str_int + '_Resampled.csv'
        df_kier_inst.to_csv(str_dirName_h + str_file_inst)
        continue
    elif str_int == '30T' : str_int = '30MIN'

    str_file_accu = 'KIER_' + str_domain + '_Resample_ACCU_' + str_int + '.csv'
    df_kier_accu = pd.read_csv(str_dirName_h + str_file_accu, index_col = 0)

    df_kier_inst = pd.DataFrame()
    for HID in list(df_kier_hList['HOUSE_ID']):
        print(HID)
        df_kier_inst['METER_DATE'] = df_kier_accu['METER_DATE']
        df_kier_inst[str_col_inst + '_' + HID] = 0
        
        for i in range(0, len(df_kier_accu) - 1) : 
                df_kier_inst[str_col_inst + '_' + HID].iloc[i] = df_kier_accu[str_col_accu + '_' + HID].iloc[i + 1] - df_kier_accu[str_col_accu + '_' + HID].iloc[i]

    str_file_inst = 'KIER_' + str_domain + '_INST_' + str_int + '_Resampled.csv'
    df_kier_inst[:-1].to_csv(str_dirName_h + str_file_inst)

df_kier_accu

10T
30T
561-1-1
561-1-2
561-1-3
561-1-4
561-2-1
561-2-2
561-2-3
561-2-4
561-3-1
561-3-2
561-3-3
561-3-4
561-4-1
561-4-2
561-4-3
561-4-4
561-5-1
561-5-2
561-5-3
561-5-4
561-6-1
561-6-2
561-6-3
561-6-4
561-7-1
561-7-2
561-7-3
561-7-4
561-8-1
561-8-2
561-8-3
561-8-4
561-9-1
561-9-2
561-9-3
561-9-4
561-10-1
561-10-2
561-10-3
561-10-4
561-11-1
561-11-2
561-11-3
561-11-4
561-12-1
561-12-2
561-12-3
561-12-4
561-13-1
561-13-2
561-13-3
561-13-4
561-14-1
561-14-2
561-14-3
561-14-4
561-15-1
561-15-2
561-15-3
561-15-4
561-16-1
561-16-2
561-16-3
561-16-4
561-17-1
561-17-2
561-17-3
561-17-4
561-18-1
561-18-2
561-18-3
561-18-4
562-1-1
562-1-2
562-1-3
562-1-4
562-1-5
562-1-6
562-2-1
562-2-2
562-2-3
562-2-4
562-2-5
562-2-6
562-3-1
562-3-2
562-3-3
562-3-4
562-3-5
562-3-6
562-4-1
562-4-2
562-4-3
562-4-4
562-4-5
562-4-6
562-5-1
562-5-2
562-5-3
562-5-4
562-5-5
562-5-6
562-6-1
562-6-2
562-6-3
562-6-4
562-6-5
562-6-6
562-7-1
562-7-2
562-7-3
562-7-4
562-7-5
562-7-6
562-8-1
562-8-2
562-8-3
562-8-4
562-8-5
562-

,METER_DATE,GAS_ACCU_FLOW_561-1-1,GAS_ACCU_FLOW_561-1-2,GAS_ACCU_FLOW_561-1-3,GAS_ACCU_FLOW_561-1-4,GAS_ACCU_FLOW_561-2-1,GAS_ACCU_FLOW_561-2-2,GAS_ACCU_FLOW_561-2-3,GAS_ACCU_FLOW_561-2-4,GAS_ACCU_FLOW_561-3-1,...,GAS_ACCU_FLOW_563-22-4,GAS_ACCU_FLOW_563-22-5,GAS_ACCU_FLOW_563-22-6,GAS_ACCU_FLOW_563-23-1,GAS_ACCU_FLOW_563-23-2,GAS_ACCU_FLOW_563-23-3,GAS_ACCU_FLOW_563-23-4,GAS_ACCU_FLOW_563-24-1,GAS_ACCU_FLOW_563-24-2,None
0,2022-08-31,0.1200000000,0.1300000000,1.5400000000,0.1400000000,0.1100000000,0.6800000000,0.1100000000,1.0900000000,1.1500000000,...,0.9100000000,1.7300000000,0.9000000000,0.1000000000,0.1200000000,0.0000000000,0.0000000000,1.5300000000,0.1100000000,0
1,2022-09-30,0.1513124509,0.2984142492,2.5055268685,0.1961796558,0.1920955712,1.6134859175,0.1420623789,1.4813124509,2.4718491605,...,1.6046185313,2.7742720660,1.8249355418,0.2066235948,0.1911871953,1.8337672850,0.0325823528,2.9513124509,0.1626768962,0
2,2022-10-31,0.7535804139,1.4746275580,7.4624698871,0.8691116875,1.3552611971,5.1489358794,0.7795906433,3.7035804139,10.0892934004,...,6.1968864944,9.9948048345,4.6944662471,1.0941251230,0.9545072195,10.3801538249,14.9786923167,11.5491119282,1.3532129476,0
3,2022-11-30,0.7986373867,1.6359285527,40.2774063015,0.9634392253,39.3482253038,606176.7008290266,0.9221696814,5.3086373867,9222.8519319495,...,9.4396429471,14.3811661689,6.1077144065,1.3373140901,1.1469876669,16.7176520882,22.3907598380,1851.5971141552,1.6922624618,0
4,2022-12-31,0.8208226582,1.6498148894,44.4122368606,1.0321147596,39.4744015115,606180.8464475063,1.1352102905,7.0019427975,9231.5862054530,...,14.2286410851,18.9844108493,7.2614693654,1.5630309381,1.3486505365,21.3401653732,28.6839543988,1860.7771306734,1.8027386943,0
5,2023-01-31,0.8623695692,1.6808580427,49.8407959353,1.2603165466,39.6246801879,606186.6259715462,1.2457399624,9.0474106322,9238.8870339979,...,19.6299234317,22.5684643641,8.3514693654,1.7895212855,1.5015867578,26.6338846605,37.8121503702,1869.3996759211,1.9601075528,0
6,2023-02-28,0.8807624379,1.7055037968,54.0333222097,1.3219498489,39.7061127414,606191.1514475242,1.3175649277,10.6096735999,9245.3955643957,...,23.7053061582,27.6528110450,9.2843065325,1.8673068230,1.6332628927,31.3998530091,43.6274138393,1877.9177023927,2.0343841930,0
7,2023-03-31,0.8807624379,1.7147750447,57.5133695681,1.5112792893,39.7471366313,606195.3390900454,1.3927410738,13.7296735999,9253.0690742667,...,28.0654013321,31.2828110450,10.9143065325,1.8969208676,1.7917297985,37.0343079483,49.7007060459,1886.5142350034,2.1539751114,0
8,2023-04-30,0.8807624379,1.7497725850,61.5884332797,1.7341916926,39.7642390118,606198.9002268070,1.4929344765,17.0796735999,9259.7034681678,...,34.4672998637,36.0228110450,12.5044814601,1.9692249789,1.9301666809,42.7145695762,55.5307935099,1895.0183623954,2.2666709111,0
9,2023-05-31,0.8807624379,1.8139706005,66.3193952185,1.9514157618,39.8082197547,606201.9753942215,1.5481960660,19.4596735999,9266.8823315615,...,38.7630910626,40.0328400305,13.4453620930,2.0360209812,2.0495009006,47.2345695762,61.0627253869,1903.9619125584,2.2974667974,0


In [9]:
df_kier_inst[:-1]

,METER_DATE,GAS_INST_FLOW_561-1-1,GAS_INST_FLOW_561-1-2,GAS_INST_FLOW_561-1-3,GAS_INST_FLOW_561-1-4,GAS_INST_FLOW_561-2-1,GAS_INST_FLOW_561-2-2,GAS_INST_FLOW_561-2-3,GAS_INST_FLOW_561-2-4,GAS_INST_FLOW_561-3-1,...,GAS_INST_FLOW_563-22-3,GAS_INST_FLOW_563-22-4,GAS_INST_FLOW_563-22-5,GAS_INST_FLOW_563-22-6,GAS_INST_FLOW_563-23-1,GAS_INST_FLOW_563-23-2,GAS_INST_FLOW_563-23-3,GAS_INST_FLOW_563-23-4,GAS_INST_FLOW_563-24-1,GAS_INST_FLOW_563-24-2
0,2022-08-31,0.0313124509,0.1684142492,0.9655268685,0.0561796558,0.0820955712,0.9334859175,0.0320623789,0.3913124509,1.3218491605,...,1.1930661527,0.6946185313,1.0442720660,0.9249355418,0.1066235948,0.0711871953,1.8337672850,0.0325823528,1.4213124509,0.0526768962
1,2022-09-30,0.6022679630,1.1762133088,4.9569430187,0.6729320317,1.1631656259,3.5354499618,0.6375282644,2.2222679630,7.6174442399,...,5.6162945294,4.5922679630,7.2205327685,2.8695307054,0.8875015282,0.7633200242,8.5463865399,14.9461099640,8.5977994773,1.1905360514
2,2022-10-31,0.0450569728,0.1613009946,32.8149364144,0.0943275378,37.9929641067,606171.5518931473,0.1425790381,1.6050569728,9212.7626385491,...,5.6870270040,3.2427564527,4.3863613344,1.4132481594,0.2431889671,0.1924804474,6.3374982632,7.4120675212,1840.0480022271,0.3390495142
3,2022-11-30,0.0221852715,0.0138863367,4.1348305591,0.0686755342,0.1261762077,4.1456184797,0.2130406091,1.6933054108,8.7342735035,...,6.3356049232,4.7889981380,4.6032446805,1.1537549589,0.2257168481,0.2016628695,4.6225132851,6.2931945609,9.1800165182,0.1104762325
4,2022-12-31,0.0415469110,0.0310431533,5.4285590747,0.2282017870,0.1502786764,5.7795240398,0.1105296719,2.0454678346,7.3008285449,...,5.0411071081,5.4012823466,3.5840535148,1.0900000000,0.2264903473,0.1529362214,5.2937192873,9.1281959714,8.6225452477,0.1573688584
5,2023-01-31,0.0183928687,0.0246457541,4.1925262744,0.0616333023,0.0814325535,4.5254759780,0.0718249653,1.5622629677,6.5085303978,...,5.7169490703,4.0753827265,5.0843466809,0.9328371671,0.0777855375,0.1316761349,4.7659683486,5.8152634691,8.5180264716,0.0742766402
6,2023-02-28,0.0000000000,0.0092712480,3.4800473584,0.1893294404,0.0410238899,4.1876425212,0.0751761460,3.1200000000,7.6735098710,...,6.4641824758,4.3600951739,3.6300000000,1.6300000000,0.0296140446,0.1584669058,5.6344549392,6.0732922065,8.5965326107,0.1195909184
7,2023-03-31,0.0000000000,0.0349975403,4.0750637116,0.2229124034,0.0171023805,3.5611367616,0.1001934027,3.3500000000,6.6343939011,...,6.1774520736,6.4018985316,4.7400000000,1.5901749276,0.0723041113,0.1384368824,5.6802616279,5.8300874641,8.5041273920,0.1126957996
8,2023-04-30,0.0000000000,0.0641980155,4.7309619388,0.2172240691,0.0439807429,3.0751674145,0.0552615895,2.3800000000,7.1788633937,...,6.0338404696,4.2957911989,4.0100289855,0.9408806330,0.0667960023,0.1193342197,4.5200000000,5.5319318770,8.9435501630,0.0307958864
9,2023-05-31,0.0000000000,0.1854281351,4.9751196783,0.1962597692,0.0994106699,2.5185401228,0.0165543471,2.3800000000,6.6601911445,...,4.5053976367,4.0431550826,5.2000000000,1.4200000000,0.0192791806,0.0537755638,5.9500000000,5.7759090610,8.0520991254,0.0553525460
